In [ ]:
# rss-feed-and-analyse
# dieses skript importiert den rss-feed eines blogs in unseren blog
# tbi = to be implemented

In [ ]:
# Importe
from bs4 import BeautifulSoup
import urllib.request
from urllib.error import HTTPError
from wordpress_xmlrpc import Client, WordPressPost
from wordpress_xmlrpc.methods.taxonomies import *
from wordpress_xmlrpc.methods.posts import *
from wordpress_xmlrpc.methods.users import *
from wordpress_xmlrpc.methods import *
from prettytable import PrettyTable
import nltk
import re
import html

In [ ]:
# DATEN EINTRAGEN
########################################################################################

# Welche Domain soll es denn sein?
DOMAIN = "DOMAIN_EINTRAGEN" # http://example.com
# Domain zum Backend?
BACKEND_URL = "BACKEND_DOMAIN_EINTRAGEN" # http://backend.example.com

# Wordpress Login für Backend
wp_user = "USER" # username für Wordpress Backend
wp_pass = "PASSWORT" # passwort für Wordpress Backend

########################################################################################

In [ ]:
old_posts = []
new_posts = []

# Mit Wordpress Client verbinden

BACKEND_x_URL = BACKEND_URL + "/xmlrpc.php"
wordpressClient = Client(BACKEND_x_URL, wp_user, wp_pass)



# Function: Duplikats-Check: Wir wollen nicht den selben Artikel mehrfach im Backend.
########################################################################################
def noduplicate(candidate):
    
    if candidate in old_posts:
        return False
    
    return True
########################################################################################



# Function: Alle bestehenden Titles in old_posts ablegen
########################################################################################
i = 1

while True:
    
    try:
        
        with urllib.request.urlopen(DOMAIN + '/feed/?paged=' + str(i)) as response:
            feedXYZ = response.read()
        
        soup = BeautifulSoup(feedXYZ, 'xml')

        for item in soup.find_all('item'):

            title = item.title.string
            old_posts.append(title)
        
        i = i + 1
        
    except (HTTPError):
            break         
########################################################################################



# Schleife, um (neue) Artikel in das Backenend aufzunehmen
########################################################################################
j = 1

while True:
    
    try:
        
        with urllib.request.urlopen(DOMAIN + '/feed/?paged=' + str(j)) as response:
            smokevoteFeed = response.read()
        
        soup = BeautifulSoup(smokevoteFeed, 'xml')

        for item in soup.find_all('item'):

            pair = (item.title.string, item.encoded)
            
            if (noduplicate(str(item.title.string))):
                new_posts.append(pair)
        
        j = j + 1
        
    except (HTTPError):
            break
########################################################################################


# function: WordpressXport
########################################################################################
def WordpressXport():
    for pair in new_posts:
    
        post = WordPressPost()
        post.title = str(pair[0])
        post.content = str(pair[1]) 
        post.terms_names = {'post_tag': ['rss-export']}
        post.post_status = 'publish'
        wordpressClient.call(NewPost(post))
########################################################################################


# Funktion anstoßen
WordpressXport()
    

# Anzahl der Wörter berechnen
########################################################################################
# Algo von: https://stackoverflow.com/questions/19410018/how-to-count-the-number-of-words-in-a-sentence
def n_of_words(x):
    
    tempcount = 0  
    count = 1  

    try:  
        for character in x:  
            if character == " ":  
                tempcount +=1  
                if tempcount ==1:  
                    count +=1  

                else:  
                    tempcount +=1
            else:
                tempcount=0

        return count  

    except Exception:  
        error = "Kein String"  
        return error  
    
########################################################################################


def title_length(x):
    
    return len(x)

    
# Output-Tabelle
########################################################################################
def tableGenerate():
    
    output_items = []
    
    j = 1

    while True:
    
        try:
        
            with urllib.request.urlopen(DOMAIN + '/feed/?paged=' + str(j)) as response:
                feedXY = response.read()
        
            soup = BeautifulSoup(feedXY, 'xml')

            for item in soup.find_all('item'):

                pair = (item.title.string, item.encoded.text)
                output_items.append(pair)
        
                j = j + 1
        
        except (HTTPError):
            break
 
    
    # Tabelle erstellen und Spalten nach links formattieren
    x = PrettyTable()
    x.field_names = ["Titel", "Titellänge", "# Wörter im Title"]
    x.align["Titel"] = "l"
    x.align["Titellänge"] = "l"
    x.align["# Wörter im Title"] = "l"
    x.sortby = "Titellänge"
    x.reversesort = True
    
    for item in output_items:
        
        # textkorpus = str(item[1])
        x.add_row([str(item[0]), title_length(str(item[0])), n_of_words(str(item[0]))])
    
    print(x)
########################################################################################

tableGenerate()